In [1]:
from helper import *
from dependencies import *
from loaddata import review_data,business_data, rest_data, user_data, elite_udata, non_elite_udata

Using TensorFlow backend.


User Data Loaded
Businesses Data Loaded
Review Data Loaded


In [2]:
cv, bid_2_stars_ne, review_data_ne, bid_2_textenc = give_text_feats_tfidf(review_data)
bid_2_stars_e = review_data.loc[review_data['iselite'] == True].groupby('business_id')['stars'].apply(list).to_dict()

Size of Vocabulary:  15374


In [4]:
def create_dataset(min_thold, bdata, bid_2_stars, bid_2_stars_e, bid_2_textenc, textenc = False, vec = False, max_thold=float('inf')):
    basicprint = False
    x1 = []
    x2 = []
    y = []
    bids = bdata['business_id'].values
    for bid in bids:
        if bid in bid_2_stars and bid in bid_2_stars_e and bid in bid_2_textenc:
            stars_e = bid_2_stars_e[bid]
            stars_ne = bid_2_stars[bid]
            lestars = len(stars_e)
            if(lestars >= min_thold and lestars < max_thold):
                x = []
                x.append([Counter(stars_ne)[i] for i in range(1,6)])
                x.append([len(stars_ne)])
                x.append([np.mean(stars_ne)])
                if textenc:
                    if basicprint:
                        print('Num features before text: ', len([item for sublist in x for item in sublist]))
                    x.append(bid_2_textenc[bid])
                    if basicprint:
                        print('Num features after text: ', len([item for sublist in x for item in sublist]))
                        basicprint = False
                x = [item for sublist in x for item in sublist]
                x1.append(x)
                x = []
                if vec:
                    x.append(give_categ(np.mean(stars_ne)))
                x = [item for sublist in x for item in sublist]
                x2.append(x)
                y.append(np.mean(stars_e))                        
    x1 = np.array(x1)
    x2 = np.array(x2)
    if not vec:
        x2 = x2.reshape(-1,1)
    xfeat = np.concatenate((x1,x2),axis=1)
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    xfeat = imp.fit_transform(xfeat)
    yfeat = np.array(y).reshape(-1,)
    print("Dimension of Our Dataset: ", xfeat.shape)
    return xfeat,yfeat

In [6]:
def give_categ(avg):
    temp = np.zeros(41)
    c = int((avg-1)/0.1) 
    temp[c] = 1
    return temp

In [7]:
xfeat,yfeat= create_dataset(40,business_data,bid_2_stars_ne,bid_2_stars_e,bid_2_textenc,True,True)
noneliteavg = xfeat[:,6].reshape(-1,1)

Dimension of Our Dataset:  (5926, 15422)


In [16]:
xtrain, xtest, ytrain, ytest = give_train_test(noneliteavg, yfeat)
model  = give_reg_model_linear(0)
test_model(model,xtrain,ytrain,xtest,ytest)

Linear
MSE:  0.0408
R-Squared :  0.7683
Margin  0.10  0.20  0.30  0.40  0.50
        40.13  70.07  87.52  94.44  98.06
 


(Ridge(alpha=0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 0.7683452418124346)

In [17]:
print(model.coef_[0])
print(model.intercept_)

0.6647555039544589
1.3670049383404042


In [18]:
model = linear_model.LinearRegression()
test_model(model,xtrain,ytrain,xtest,ytest)

MSE:  0.0408
R-Squared :  0.7683
Margin  0.10  0.20  0.30  0.40  0.50
        40.13  70.07  87.52  94.44  98.06
 


(LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
          normalize=False), 0.7683452418124346)

In [19]:
print(model.coef_[0])
print(model.intercept_)

0.6647555039544589
1.3670049383404042


In [10]:
%matplotlib qt
plt.xlabel("Average Non-Elite Ratings")
plt.ylabel("Average Elite-Ratings")
plt.scatter(noneliteavg,yfeat,color='green',label='Scatter Plot')
x = [1.5,2,2.5,3,3.5,4,4.5,5]
reg_out = [model.coef_[0]*a+model.intercept_ for a in  x]
plt.plot(x,x,'r-', label='Identity Line', linewidth = 2)
plt.plot(x,reg_out,'b-', label='Linear Regression', linewidth = 2)
plt.legend(loc="best")

In [11]:
xtrain, xtest, ytrain, ytest = give_train_test(noneliteavg, yfeat)
print(ytrain.shape)
print(ytest.shape)
print_prediction(xtest,ytest.reshape(-1,1))

(4740,)
(1186,)
MSE:  0.0757
R-Squared :  0.57
Margin  0.10  0.20  0.30  0.40  0.50
        33.39  59.36  76.31  85.33  92.58
 


0.5699704082332262

In [12]:
bidswith40elites = [bid for bid,ratings in bid_2_stars_e.items() if  bid in bid_2_stars_ne and bid in bid_2_textenc and len(ratings) >= 40]
bids_elites_higher = [bid for bid,ratings in bid_2_stars_e.items() if  bid in bid_2_stars_ne and bid in bid_2_textenc and len(ratings) >= 40 and np.mean(ratings) > np.mean(bid_2_stars_ne[bid])]
bids_elites_lower = [bid for bid,ratings in bid_2_stars_e.items() if  bid in bid_2_stars_ne and bid in bid_2_textenc and len(ratings) >= 40 and np.mean(ratings) < np.mean(bid_2_stars_ne[bid])]
bids_elites_equal = [bid for bid,ratings in bid_2_stars_e.items() if  bid in bid_2_stars_ne and bid in bid_2_textenc and len(ratings) >= 40 and np.mean(ratings) == np.mean(bid_2_stars_ne[bid])]
print(len(bidswith40elites))
print(len(bids_elites_higher))
print(len(bids_elites_lower))
print(len(bids_elites_equal))


5926
3681
2240
5


In [17]:
ne_ratings = [x for bid in bids_elites_higher for x in bid_2_stars_ne[bid]]
#e_ratings = [x for bid in bids_elites_higher for x in bid_2_stars_e[bid]]
plot_ratings_dist(ne_ratings, "Rating : X", "Percentage of X star rating by non-elites")
#plot_ratings_dist(e_ratings)

[0.14354, 0.10575, 0.12897, 0.2423, 0.37944]


In [16]:
def plot_ratings_dist(t,xl="",yl=""):
    countnumberofeachstar = [Counter(t)[i] for i in range(1,6)]
    totalnumberofratings = len(t)
    distribution = [round(x/totalnumberofratings,5) for x in countnumberofeachstar]
    print(distribution)
    plt.bar([1,2,3,4,5],distribution, width=0.5)
    for a,b in zip([1,2,3,4,5], distribution):
        plt.text(a - 0.15, b + 0.004, str(np.round(b*100)))
    plt.xlabel(xl)
    plt.ylabel(yl)
    plt.show()

In [18]:
ne_ratings = [x for bid in bids_elites_lower for x in bid_2_stars_ne[bid]]
#e_ratings = [x for bid in bids_elites_lower for x in bid_2_stars_e[bid]]
plot_ratings_dist(ne_ratings, "Rating : X", "Percentage of X star rating by non-elites")
#plot_ratings_dist(e_ratings)

[0.06183, 0.06082, 0.09422, 0.23901, 0.54413]


In [ ]:
ne_ratings = [x for bid in bids_elites_equal for x in bid_2_stars_ne[bid]]
#e_ratings = [x for bid in bids_elites_equal for x in bid_2_stars_e[bid]]
plot_ratings_dist(ne_ratings, "Rating : X", "Percentage of X star rating by non-elites")
#plot_ratings_dist(e_ratings)

[0.06832, 0.06832, 0.13665, 0.29814, 0.42857]


In [ ]:
e_ratings = [x for bid in bidswith40elites for x in bid_2_stars_e[bid]]
plot_ratings_dist(e_ratings, "Rating : X", "Percentage of X star rating by elites")

[0.02469, 0.06895, 0.19631, 0.39761, 0.31245]


In [ ]:
ne_ratings = [x for bid in bidswith40elites for x in bid_2_stars_ne[bid]]
plot_ratings_dist(ne_ratings, "Rating : X", "Percentage of X star rating by non-elites")

[0.11151, 0.08814, 0.11536, 0.24102, 0.44397]
